# Distirbuted Training of Mask-RCNN in Amazon SageMaker using S3

This notebook is a step-by-step tutorial on distributed tranining of [Mask R-CNN](https://arxiv.org/abs/1703.06870) implemented in [TensorFlow](https://www.tensorflow.org/) framework. Mask R-CNN is also referred to as heavy weight object detection model and it is part of [MLPerf](https://www.mlperf.org/training-results-0-6/).

Concretely, we will describe the steps for training [TensorPack Faster-RCNN/Mask-RCNN](https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN) and [AWS Samples Mask R-CNN](https://github.com/aws-samples/mask-rcnn-tensorflow) in [Amazon SageMaker](https://aws.amazon.com/sagemaker/) using [Amazon S3](https://aws.amazon.com/s3/) as data source.

The outline of steps is as follows:

1. Stage COCO 2017 dataset in [Amazon S3](https://aws.amazon.com/s3/)
2. Build SageMaker training image and push it to [Amazon ECR](https://aws.amazon.com/ecr/)
3. Configure data input channels
4. Configure hyper-prarameters
5. Define training metrics
6. Define training job and start training

Before we get started, let us initialize two python variables ```aws_region``` and ```s3_bucket``` that we will use throughout the notebook:

In [1]:
aws_region =  "ap-southeast-1"
s3_bucket  =  "smart-invoice"

## Stage COCO 2017 dataset in Amazon S3

We use [COCO 2017 dataset](http://cocodataset.org/#home) for training. We download COCO 2017 training and validation dataset to this notebook instance, extract the files from the dataset archives, and upload the extracted files to your Amazon [S3 bucket](https://docs.aws.amazon.com/en_pv/AmazonS3/latest/gsg/CreatingABucket.html) with the prefix ```mask-rcnn/sagemaker/input/train```. The ```prepare-s3-bucket.sh``` script executes this step.


In [ ]:
!cat ./prepare-s3-bucket.sh

 Using your *Amazon S3 bucket* as argument, run the cell below. If you have already uploaded COCO 2017 dataset to your Amazon S3 bucket *in this AWS region*, you may skip this step. The expected time to execute this step is 20 minutes.

In [ ]:
%%time
!./prepare-s3-bucket.sh {s3_bucket}

## Build and push SageMaker training images

For this step, the [IAM Role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) attached to this notebook instance needs full access to Amazon ECR service. If you created this notebook instance using the ```./stack-sm.sh``` script in this repository, the IAM Role attached to this notebook instance is already setup with full access to ECR service. 

Below, we have a choice of two different implementations:

1. [TensorPack Faster-RCNN/Mask-RCNN](https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN) implementation supports a maximum per-GPU batch size of 1, and does not support mixed precision. It can be used with mainstream TensorFlow releases.

2. [AWS Samples Mask R-CNN](https://github.com/aws-samples/mask-rcnn-tensorflow) is an optimized implementation that supports a maximum batch size of 4 and supports mixed precision. This implementation uses TensorFlow base version 1.13 augmented with custom TensorFlow ops. 

It is recommended that you build and push both SageMaker training images and use either image for training later.


### TensorPack Faster-RCNN/Mask-RCNN

Use ```./container/build_tools/build_and_push.sh``` script to build and push the TensorPack Faster-RCNN/Mask-RCNN  training image to Amazon ECR. 

In [ ]:
!cat ./container/build_tools/build_and_push.sh

Using your *AWS region* as argument, run the cell below.

In [ ]:
%%time
! ./container/build_tools/build_and_push.sh {aws_region}

Set ```tensorpack_image``` below to Amazon ECR URI of the image you pushed above.

In [ ]:
tensorpack_image = "393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/mask-rcnn-tensorpack-sagemaker:tf1.13-tp26664c3"

### AWS Samples Mask R-CNN
Use ```./container-optimized/build_tools/build_and_push.sh``` script to build and push the AWS Samples Mask R-CNN training image to Amazon ECR.

In [ ]:
!cat ./container-optimized/build_tools/build_and_push.sh

Using your *AWS region* as argument, run the cell below.

In [ ]:
%%time
! ./container-optimized/build_tools/build_and_push.sh {aws_region}

 Set ```aws_samples_image``` below to Amazon ECR URI of the image you pushed above.

In [2]:
aws_samples_image = "393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/mask-rcnn-tensorflow-sagemaker:tf1.13-153442b"

## SageMaker Initialization 
We have staged the data and we have built and pushed the training docker image to Amazon ECR. Now we are ready to start using Amazon SageMaker.

In [3]:
%%time
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

role = get_execution_role() # provide a pre-existing role ARN as an alternative to creating a new role
print(f'SageMaker Execution Role:{role}')

client = boto3.client('sts')
account = client.get_caller_identity()['Account']
print(f'AWS account:{account}')

session = boto3.session.Session()
region = session.region_name
print(f'AWS region:{region}')

SageMaker Execution Role:arn:aws:iam::393782509758:role/service-role/AmazonSageMaker-ExecutionRole-20191115T140457
AWS account:393782509758
AWS region:ap-southeast-1
CPU times: user 782 ms, sys: 79.1 ms, total: 861 ms
Wall time: 1.76 s


Next, we set ```training_image``` to the Amazon ECR image URI you saved in a previous step. 

In [4]:
training_image =  aws_samples_image# set to tensorpack_image or aws_samples_image 
print(f'Training image: {training_image}')

Training image: 393782509758.dkr.ecr.ap-southeast-1.amazonaws.com/mask-rcnn-tensorflow-sagemaker:tf1.13-153442b


## Define SageMaker Data Channels
In this step, we define SageMaker *train* data channel. 

In [5]:
prefix = "mask-rcnn/sagemaker" #prefix in your S3 bucket

s3train = f's3://{s3_bucket}/{prefix}/input/test'


train = sagemaker.session.s3_input(s3train, distribution='FullyReplicated', 
                        content_type='application/tfrecord', s3_data_type='S3Prefix')


data_channels = {'train': train}

Next, we define the model output location in S3 bucket.

In [6]:
s3_output_location = f's3://{s3_bucket}/{prefix}/output'

# Configure Hyper-parameters
Next, we define the hyper-parameters. 

Note, some hyper-parameters are different between the two implementations. The batch size per GPU in TensorPack Faster-RCNN/Mask-RCNN is fixed at 1, but is configurable in AWS Samples Mask-RCNN. The learning rate schedule is specified in units of steps in TensorPack Faster-RCNN/Mask-RCNN, but in epochs in AWS Samples Mask-RCNN.

The detault learning rate schedule values shown below correspond to training for a total of 24 epochs, at 120,000 images per epoch.

<table align='left'>
    <caption>TensorPack Faster-RCNN/Mask-RCNN  Hyper-parameters</caption>
    <tr>
    <th style="text-align:center">Hyper-parameter</th>
    <th style="text-align:center">Description</th>
    <th style="text-align:center">Default</th>
    </tr>
    <tr>
        <td style="text-align:center">mode_fpn</td>
        <td style="text-align:left">Flag to indicate use of Feature Pyramid Network (FPN) in the Mask R-CNN model backbone</td>
        <td style="text-align:center">"True"</td>
    </tr>
     <tr>
        <td style="text-align:center">mode_mask</td>
        <td style="text-align:left">A value of "False" means Faster-RCNN model, "True" means Mask R-CNN moodel</td>
        <td style="text-align:center">"True"</td>
    </tr>
     <tr>
        <td style="text-align:center">eval_period</td>
        <td style="text-align:left">Number of epochs period for evaluation during training</td>
        <td style="text-align:center">1</td>
    </tr>
    <tr>
        <td style="text-align:center">lr_schedule</td>
        <td style="text-align:left">Learning rate schedule in training steps</td>
        <td style="text-align:center">'[240000, 320000, 360000]'</td>
    </tr>
    <tr>
        <td style="text-align:center">batch_norm</td>
        <td style="text-align:left">Batch normalization option ('FreezeBN', 'SyncBN', 'GN', 'None') </td>
        <td style="text-align:center">'FreezeBN'</td>
    </tr>
    <tr>
        <td style="text-align:center">images_per_epoch</td>
        <td style="text-align:left">Images per epoch </td>
        <td style="text-align:center">120000</td>
    </tr>
    <tr>
        <td style="text-align:center">data_train</td>
        <td style="text-align:left">Training data under data directory</td>
        <td style="text-align:center">'coco_train2017'</td>
    </tr>
    <tr>
        <td style="text-align:center">data_val</td>
        <td style="text-align:left">Validation data under data directory</td>
        <td style="text-align:center">'coco_val2017'</td>
    </tr>
    <tr>
        <td style="text-align:center">resnet_arch</td>
        <td style="text-align:left">Must be 'resnet50' or 'resnet101'</td>
        <td style="text-align:center">'resnet50'</td>
    </tr>
    <tr>
        <td style="text-align:center">backbone_weights</td>
        <td style="text-align:left">Pre-trained model weights</td>
        <td style="text-align:center">'ImageNet-R50-AlignPadding.npz'</td>
    </tr>
</table>

    
<table align='left'>
    <caption>AWS Samples Mask-RCNN  Hyper-parameters</caption>
    <tr>
    <th style="text-align:center">Hyper-parameter</th>
    <th style="text-align:center">Description</th>
    <th style="text-align:center">Default</th>
    </tr>
    <tr>
        <td style="text-align:center">mode_fpn</td>
        <td style="text-align:left">Flag to indicate use of Feature Pyramid Network (FPN) in the Mask R-CNN model backbone</td>
        <td style="text-align:center">"True"</td>
    </tr>
     <tr>
        <td style="text-align:center">mode_mask</td>
        <td style="text-align:left">A value of "False" means Faster-RCNN model, "True" means Mask R-CNN moodel</td>
        <td style="text-align:center">"True"</td>
    </tr>
     <tr>
        <td style="text-align:center">eval_period</td>
        <td style="text-align:left">Number of epochs period for evaluation during training</td>
        <td style="text-align:center">1</td>
    </tr>
    <tr>
        <td style="text-align:center">lr_epoch_schedule</td>
        <td style="text-align:left">Learning rate schedule in epochs</td>
        <td style="text-align:center">'[(16, 0.1), (20, 0.01), (24, None)]'</td>
    </tr>
    <tr>
        <td style="text-align:center">batch_size_per_gpu</td>
        <td style="text-align:left">Batch size per gpu ( Minimum 1, Maximum 4)</td>
        <td style="text-align:center">4</td>
    </tr>
    <tr>
        <td style="text-align:center">batch_norm</td>
        <td style="text-align:left">Batch normalization option ('FreezeBN', 'SyncBN', 'GN', 'None') </td>
        <td style="text-align:center">'FreezeBN'</td>
    </tr>
    <tr>
        <td style="text-align:center">images_per_epoch</td>
        <td style="text-align:left">Images per epoch </td>
        <td style="text-align:center">120000</td>
    </tr>
    <tr>
        <td style="text-align:center">data_train</td>
        <td style="text-align:left">Training data under data directory</td>
        <td style="text-align:center">'train2017'</td>
    </tr>
    <tr>
        <td style="text-align:center">data_val</td>
        <td style="text-align:left">Validation data under data directory</td>
        <td style="text-align:center">'val2017'</td>
    </tr>
    <tr>
        <td style="text-align:center">resnet_arch</td>
        <td style="text-align:left">Must be 'resnet50' or 'resnet101'</td>
        <td style="text-align:center">'resnet50'</td>
    </tr>
    <tr>
        <td style="text-align:center">backbone_weights</td>
        <td style="text-align:left">Pre-trained model weights</td>
        <td style="text-align:center">'ImageNet-R50-AlignPadding.npz'</td>
    </tr>
</table>

In [15]:
hyperparameters = {
                    "mode_fpn": "True",
                    "mode_mask": "True",
                    "eval_period": 1,
                    "batch_norm": "FreezeBN",
                    "batch_size_per_gpu": 1,
                    "images_per_epoch": 1
                  }

## Define Training Metrics
Next, we define the regular expressions that SageMaker uses to extract algorithm metrics from training logs and send them to [AWS CloudWatch metrics](https://docs.aws.amazon.com/en_pv/AmazonCloudWatch/latest/monitoring/working_with_metrics.html). These algorithm metrics are visualized in SageMaker console.

In [16]:
metric_definitions=[
             {
                "Name": "fastrcnn_losses/box_loss",
                "Regex": ".*fastrcnn_losses/box_loss:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/label_loss",
                "Regex": ".*fastrcnn_losses/label_loss:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/label_metrics/accuracy",
                "Regex": ".*fastrcnn_losses/label_metrics/accuracy:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/label_metrics/false_negative",
                "Regex": ".*fastrcnn_losses/label_metrics/false_negative:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/label_metrics/fg_accuracy",
                "Regex": ".*fastrcnn_losses/label_metrics/fg_accuracy:\\s*(\\S+).*"
            },
            {
                "Name": "fastrcnn_losses/num_fg_label",
                "Regex": ".*fastrcnn_losses/num_fg_label:\\s*(\\S+).*"
            },
             {
                "Name": "maskrcnn_loss/accuracy",
                "Regex": ".*maskrcnn_loss/accuracy:\\s*(\\S+).*"
            },
            {
                "Name": "maskrcnn_loss/fg_pixel_ratio",
                "Regex": ".*maskrcnn_loss/fg_pixel_ratio:\\s*(\\S+).*"
            },
            {
                "Name": "maskrcnn_loss/maskrcnn_loss",
                "Regex": ".*maskrcnn_loss/maskrcnn_loss:\\s*(\\S+).*"
            },
            {
                "Name": "maskrcnn_loss/pos_accuracy",
                "Regex": ".*maskrcnn_loss/pos_accuracy:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/IoU=0.5",
                "Regex": ".*mAP\\(bbox\\)/IoU=0\\.5:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/IoU=0.5:0.95",
                "Regex": ".*mAP\\(bbox\\)/IoU=0\\.5:0\\.95:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/IoU=0.75",
                "Regex": ".*mAP\\(bbox\\)/IoU=0\\.75:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/large",
                "Regex": ".*mAP\\(bbox\\)/large:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/medium",
                "Regex": ".*mAP\\(bbox\\)/medium:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(bbox)/small",
                "Regex": ".*mAP\\(bbox\\)/small:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/IoU=0.5",
                "Regex": ".*mAP\\(segm\\)/IoU=0\\.5:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/IoU=0.5:0.95",
                "Regex": ".*mAP\\(segm\\)/IoU=0\\.5:0\\.95:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/IoU=0.75",
                "Regex": ".*mAP\\(segm\\)/IoU=0\\.75:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/large",
                "Regex": ".*mAP\\(segm\\)/large:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/medium",
                "Regex": ".*mAP\\(segm\\)/medium:\\s*(\\S+).*"
            },
            {
                "Name": "mAP(segm)/small",
                "Regex": ".*mAP\\(segm\\)/small:\\s*(\\S+).*"
            }  
            
    ]

## Define SageMaker Training Job

Next, we use SageMaker [Estimator](https://sagemaker.readthedocs.io/en/stable/estimators.html) API to define a SageMaker Training Job. 

We recommned using 32 GPUs, so we set ```train_instance_count=4``` and ```train_instance_type='ml.p3.16xlarge'```, because there are 8 Tesla V100 GPUs per ```ml.p3.16xlarge``` instance. We recommend using 100 GB [Amazon EBS](https://aws.amazon.com/ebs/) storage volume with each training instance, so we set ```train_volume_size = 100```. We want to replicate training data to each training instance, so we set ```input_mode= 'File'```.

We run the training job in your private VPC, so we need to set the ```subnets``` and ```security_group_ids``` prior to running the cell below. You may specify multiple subnet ids in the ```subnets``` list. The subnets included in the ```sunbets``` list must be part of the output of  ```./stack-sm.sh``` CloudFormation stack script used to create this notebook instance. Specify only one security group id in ```security_group_ids``` list. The security group id must be part of the output of  ```./stack-sm.sh``` script.

For ```train_instance_type``` below, you have the option to use ```ml.p3.16xlarge``` with 16 GB per-GPU memory and 25 Gbs network interconnectivity, or ```ml.p3dn.24xlarge``` with 32 GB per-GPU memory and 100 Gbs network interconnectivity. The ```ml.p3dn.24xlarge``` instance type offers significantly better performance than ```ml.p3.16xlarge``` for Mask R-CNN distributed TensorFlow training.

In [23]:
# security_group_ids =  # ['sg-xxxxxxxx']
# subnets =      # [ 'subnet-xxxxxxx']
sagemaker_session = sagemaker.session.Session(boto_session=session)
mask_rcnn_estimator = Estimator(training_image,
                                         role, 
                                         train_instance_count=1, 
                                         train_instance_type='ml.p3.2xlarge',
                                         #train_instance_type='local_gpu',
                                         train_volume_size = 50,
                                         train_max_run = 100,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sagemaker_session, 
                                         hyperparameters = hyperparameters,
                                         metric_definitions = metric_definitions,
                                         base_job_name="mask-rcnn-s3")



Finally, we launch the SageMaker training job. 

The estimated time for downloading data to all the training instances is 20 minutes. The time to complete the training depends on type and number of training instances, and the training image used for training.

In [24]:
mask_rcnn_estimator.fit(inputs=data_channels, logs="All")

2020-04-02 10:49:58 Starting - Starting the training job...
2020-04-02 10:49:59 Starting - Launching requested ML instances......
2020-04-02 10:51:00 Starting - Preparing the instances for training......
2020-04-02 10:52:11 Downloading - Downloading input data
2020-04-02 10:52:11 Training - Downloading the training image............
2020-04-02 10:54:23 Training - Training image download completed. Training in progress../usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.ui

[1,0]<stdout>:#033[0402 10:54:36 @registry.py:135]#033 group1/block2/conv3 output: [None, 512, None, None]
[1,0]<stdout>:#033[0402 10:54:36 @registry.py:127]#033 group1/block3/conv1 input: [None, 512, None, None]
[1,0]<stdout>:#033[0402 10:54:36 @batch_norm.py:166]#033 #033#033WRN#033 [BatchNorm] Using moving_mean/moving_variance in training.
[1,0]<stdout>:#033[0402 10:54:36 @registry.py:135]#033 group1/block3/conv1 output: [None, 128, None, None]
[1,0]<stdout>:#033[0402 10:54:36 @registry.py:127]#033 group1/block3/conv2 input: [None, 128, None, None]
[1,0]<stdout>:#033[0402 10:54:36 @batch_norm.py:166]#033 #033#033WRN#033 [BatchNorm] Using moving_mean/moving_variance in training.
[1,0]<stdout>:#033[0402 10:54:36 @registry.py:135]#033 group1/block3/conv2 output: [None, 128, None, None]
[1,0]<stdout>:#033[0402 10:54:36 @registry.py:127]#033 group1/block3/conv3 input: [None, 128, None, None]
[1,0]<stdout>:#033[0402 10:54:36 @batch_norm.py:166]#033 #033#033WRN#033 [BatchNorm] Using moving

[1,0]<stdout>:#033[0402 10:54:47 @model_utils.py:66]#033 #033Trainable Variables:
[1,0]<stdout>:#033name                                   shape                    dim
[1,0]<stdout>:-------------------------------------  ------------------  --------
[1,0]<stdout>:group1/block0/conv1/W:0                [1, 1, 256, 128]       32768
[1,0]<stdout>:group1/block0/conv1/bn/gamma:0         [128]                    128
[1,0]<stdout>:group1/block0/conv1/bn/beta:0          [128]                    128
[1,0]<stdout>:group1/block0/conv2/W:0                [3, 3, 128, 128]      147456
[1,0]<stdout>:group1/block0/conv2/bn/gamma:0         [128]                    128
[1,0]<stdout>:group1/block0/conv2/bn/beta:0          [128]                    128
[1,0]<stdout>:group1/block0/conv3/W:0                [1, 1, 128, 512]       65536
[1,0]<stdout>:group1/block0/conv3/bn/gamma:0         [512]                    512
[1,0]<stdout>:group1/block0/conv3/bn/beta:0          [512]                    512
[1,0]<stdout

[1,0]<stdout>:#033[0402 10:54:57 @base.py:231]#033 Creating the session ...
[1,0]<stdout>:#033[0402 10:55:03 @base.py:237]#033 Initializing the session ...
[1,0]<stdout>:#033[0402 10:55:03 @sessinit.py:206]#033 Variables to restore from dict: group2/block4/conv3/bn/beta:0, group2/block4/conv2/bn/gamma:0, group1/block2/conv3/bn/variance/EMA:0, group0/block0/conv1/bn/variance/EMA:0, group2/block3/conv2/bn/beta:0, group2/block4/conv3/bn/gamma:0, group2/block4/conv3/bn/variance/EMA:0, group1/block0/conv2/bn/mean/EMA:0, group2/block3/conv2/bn/variance/EMA:0, group1/block0/convshortcut/bn/beta:0, group2/block5/conv3/bn/variance/EMA:0, group3/block2/conv2/bn/variance/EMA:0, group0/block0/conv3/W:0, group2/block0/convshortcut/bn/gamma:0, group3/block1/conv1/bn/variance/EMA:0, group0/block0/conv3/bn/gamma:0, group1/block2/conv1/bn/beta:0, group2/block0/conv1/bn/mean/EMA:0, group0/block0/conv1/W:0, group3/block0/conv1/W:0, group1/block3/conv3/W:0, group0/block0/conv3/bn/variance/EMA:0, group1/bl

[1,0]<stdout>:#033[0402 10:55:08 @varmanip.py:104]#033 #033#033WRN#033 Variable group0/block0/convshortcut/W has dtype <dtype: 'float16'> but was given a value of dtype float32. Load it after downcasting!
[1,0]<stdout>:#033[0402 10:55:08 @varmanip.py:104]#033 #033#033WRN#033 Variable group0/block1/conv2/W has dtype <dtype: 'float16'> but was given a value of dtype float32. Load it after downcasting!
[1,0]<stdout>:#033[0402 10:55:11 @varmanip.py:104]#033 #033#033WRN#033 Variable group0/block2/conv3/W has dtype <dtype: 'float16'> but was given a value of dtype float32. Load it after downcasting!
[1,0]<stdout>:#033[0402 10:55:13 @varmanip.py:104]#033 #033#033WRN#033 Variable group0/block0/conv2/W has dtype <dtype: 'float16'> but was given a value of dtype float32. Load it after downcasting!
[1,0]<stdout>:#033[0402 10:55:13 @varmanip.py:104]#033 #033#033WRN#033 Variable conv0/W has dtype <dtype: 'float16'> but was given a value of dtype float32. Load it after downcasting!
[1,0]<stdout>:#03

[1,0]<stdout>:loading annotations into memory...
[1,0]<stdout>:Done (t=0.00s)
[1,0]<stdout>:creating index...
[1,0]<stdout>:index created!
[1,0]<stdout>:#033[0402 10:56:04 @dataset.py:50]#033 Instances loaded from /opt/ml/input/data/train/annotations/instances_val2017.json.
[1,0]<stdout>:#033[0402 10:56:04 @performance.py:179]#033 [ThroughputTracker] Over last epoch, MeanEpochThroughput: 10.71
[1,0]<stdout>:#033[0402 10:56:04 @performance.py:180]#033 [ThroughputTracker] Over last epoch, EpochWallClockDuration: 0.09
[1,0]<stdout>:#033[0402 10:56:04 @performance.py:181]#033 [ThroughputTracker] Over last epoch, CallbackOverheadDuration: 0.00
[1,0]<stdout>:#033[0402 10:56:04 @monitor.py:469]#033 PeakMemory(MB)/gpu:0: 4585
[1,0]<stdout>:#033[0402 10:56:04 @monitor.py:469]#033 QueueInput/queue_size: 50
[1,0]<stdout>:#033[0402 10:56:04 @monitor.py:469]#033 Throughput/CallbackOverheadDuration: 4.2439e-05
[1,0]<stdout>:#033[0402 10:56:04 @monitor.py:469]#033 Throughput/EpochWallClockDuration: 0


2020-04-02 10:56:09 Uploading - Uploading generated training model
2020-04-02 10:57:57 Completed - Training job completed
Training seconds: 358
Billable seconds: 358
